# Week 3: Spectral Properties of LFP

This notebook explores spectral analysis methods for local field potential (LFP) signals.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy import signal